In [1]:
#Explore the AAPL Data

SELECT
  MIN(date) AS min_date,
  MAX(date) AS max_date
FROM
  `ai4f.AAPL10Y`

IndentationError: unexpected indent (<ipython-input-1-e3feeb60cc11>, line 4)

In [2]:
# Copy and paste the following SQL code into the Query Editor:
SELECT
  EXTRACT(year FROM date) AS year,
  AVG(close) AS avg_close
FROM
  `ai4f.AAPL10Y`
GROUP BY
  year
ORDER BY
  year DESC

IndentationError: unexpected indent (<ipython-input-2-528862183cdc>, line 3)

In [ ]:
# Question: Which five dates correspond to the greatest percent increases in AAPL stock?
# Copy and paste the following SQL code into the Query Editor:
    
SELECT
  date,
  100.0 * close / LAG(close, 1) OVER(ORDER BY date) AS pct_close_change
FROM
  `ai4f.AAPL10Y`
ORDER BY
  pct_close_change DESC
LIMIT
  5

In [ ]:
# Build a Linear Regression Model in BigQuery

WITH
  raw AS (
  SELECT
    date,
    close,
    LAG(close, 1) OVER(ORDER BY date) AS min_1_close,
    LAG(close, 2) OVER(ORDER BY date) AS min_2_close,
    LAG(close, 3) OVER(ORDER BY date) AS min_3_close,
    LAG(close, 4) OVER(ORDER BY date) AS min_4_close
  FROM
    `ai4f.AAPL10Y`
  ORDER BY
    date DESC ),
  raw_plus_trend AS (
  SELECT
    date,
    close,
    min_1_close,
    IF (min_1_close - min_2_close > 0, 1, -1) AS min_1_trend,
    IF (min_2_close - min_3_close > 0, 1, -1) AS min_2_trend,
    IF (min_3_close - min_4_close > 0, 1, -1) AS min_3_trend
  FROM
    raw ),
  ml_data AS (
  SELECT
    date,
    close,
    min_1_close AS day_prev_close,
    IF (min_1_trend + min_2_trend + min_3_trend > 0, 1, -1) AS trend_3_day
  FROM
    raw_plus_trend )
SELECT
  *
FROM
  ml_data


In [3]:
# model_data

CREATE OR REPLACE MODEL `ai4f.aapl_model`
OPTIONS
  ( model_type='linear_reg',
    input_label_cols=['close'],
    data_split_method='seq',
    data_split_eval_fraction=0.3,
    data_split_col='date') AS
SELECT
  date,
  close,
  day_prev_close,
  trend_3_day
FROM
  `ai4f.model_data`

SyntaxError: invalid syntax (<ipython-input-3-f1145252583b>, line 3)

In [4]:
# Evaluate Regression Model Performance

SELECT * FROM ML.EVALUATE(MODEL `ai4f.aapl_model`)

SyntaxError: invalid syntax (<ipython-input-4-80593b710aaa>, line 3)

In [5]:
# Make Predictions Using Model

SELECT
  *
FROM
  ml.PREDICT(MODEL `ai4f.aapl_model`,
    (
    SELECT
      *
    FROM
      `ai4f.model_data`
    WHERE
      date >= '2019-01-01') )

IndentationError: unexpected indent (<ipython-input-5-579609af42f7>, line 4)